In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget
layout = widgets.Layout(align_items = 'center')

<h2> REGRESIÓN LOGÍSTICA: CLASIFICACIÓN </h2>

La regresión logística busca predecir un valor de salida entre un 
número pequeño de valores discretos de salida posibles.
Cuando la salida solo puede tomar 2 valores ($y \in \{0,1\}$), el 
método se denomina <b>clasificación binaria</b>. Cuando la salida 
puede tomar un mayor número de valores, el método se denomina 
<b>clasificación multiclase</b>, o también <b>clasificación uno 
    contra todos</b>.

<h3>CLASIFICACIÓN BINARIA</h3>
La función hipótesis de regresión logística binaria es de la forma:
            $$
                h_\theta(x^i)=g(\theta_0+\theta_1x_1^i)
            $$
            $$
                h_\theta(x^i)=g(\theta_0+\theta_1x_1^i+\theta_2x_2^i+...+\theta_nx_n^i)
            $$
<br>    
<ul>   
     <li>g(z) es denominada <b> función sigmoide</b>: Su función es 
         mapear cualquier número real al intervalo (0,1), de manera 
         que permite transformar una función de valores arbitrarios 
         a otra función que permite la clasificación.
          $$
              g(z)=\frac{1}{1+e^{-z}}
          $$</li>
<br>  
    <li>$h_\theta(x)$ da como resultado un valor de <b>probabilidad</b> 
        de que la salida sea 1. La probabilidad de que la predicción 
        sea 0 es el complemento de la probabilidad de que sea 1.
        $$
            h_\theta(x)= P(y=1|x;\theta)= 1-P(y=0|x;\theta) 
        $$
        $$
            P(y=1|x;\theta)+P(y=0|x;\theta)=1
        $$</li>
<br>     
    <li><strong>UMBRAL DE DECISIÓN</strong> es la línea que separa el 
        área donde $y=0$ y $y=1$, permitiendo obtener la clasificación 
        discreta a partir de la función de hipótesis.
        $$
            h_\theta(x)\geq 0.5 \to y=1
        $$
        $$
            h_\theta(x) < 0.5 \to y=0
        $$
        Según el comportamiento de la función $g(z)$:
        $$
            z\geq 0 \to g(z)\geq 0 \to h_\theta(x)\geq 0.5 
        $$
    </li> 
</ul>


In [ ]:
z_s = np.arange(-8, 8, 0.1) 

out_sigmoid = widgets.Output()
with out_sigmoid:
    fig_s, ax_s = plt.subplots(figsize = (8,4),tight_layout = True)
    fig_s.suptitle('Función Sigmoide')
    ax_s.plot(z_s, 1 / (1 + np.exp(-z_s)), 'r')
    ax_s.axhline(y = 0.5, xmin = 0.0, xmax = 1.0)
    ax_s.set_xlabel('z')
    ax_s.set_ylabel('g(z)')  
    ax_s.grid(True);

widgets.VBox([out_sigmoid],layout = layout)


<h3>FUNCIÓN DE COSTO</h3>

Si se utiliza la misma función de costos que para la regresión lineal, 
$J(\theta)$ tendrá muchos mínimos locales, dificultando la selección 
del mínimo global. Es por ello que para regresión logística se utiliza 
otra función de costo dada por:
 
$$
    J(\theta)=\frac{1}{m}\sum_{i_1}^m cost(h_\theta(x^i),y^i)
$$

$$
    cost(h_\theta(x^i),y^i)= -\log(h_\theta(x^i)) \leftrightarrow y=1
$$
$$
    cost(h_\theta(x^i),y^i)= -\log(1-h_\theta(x^i)) \leftrightarrow  y=0
$$
<center>
<img src="./figures/costFunctionLogistic.png" height="300" width="300"/>
</center>
<br>  
Estos dos casos condicionales pueden comprimirse en un único caso, de 
manera que la función costo se define como:  

$$
    J(\theta)=-\frac{1}{m}\sum_{i_1}^m \left[y^i\log(h_\theta(x^i))+(1-y^i)\log(1-h_\theta(x^i))\right]
$$


In [ ]:
x = np.arange(0, 15, 0.5)
y = (x >= 10) * 1
m = x.size

theta0_slider = widgets.IntSlider(min = -20,max = 20,value = -10,description = r'$\theta_0$')
theta1_slider = widgets.IntSlider(min = -20,max = 20,value = 1,description = r'$\theta_1$')

def plot_h(x,t0,t1):
    z =  x*t1 + t0
    h = 1/(1+np.exp(-z))
    h = (h >= 0.5) * 1
    return h
    
J1 = np.array([])
J2 = np.array([])
theta0 = np.arange(theta0_slider.min, theta0_slider.max, 0.1)

for theta in theta0: 
  z =  x + theta
  h = 1/(1+np.exp(-z))
  logH = np.nan_to_num(np.log(h)) 
  log1_H = np.nan_to_num(np.log(1-h))
  J1 = np.append(J1, 1/(2*m) * np.sum(np.square(h - y)))
  J2 = np.append(J2, -1/m * np.sum(y * logH + (1 - y) * log1_H))
    
plots_Jlr = widgets.Output()

with plots_Jlr:
    fig_Jlr, axs_Jlr = plt.subplots(1,2,figsize = (10,4),tight_layout = True)
    
    axs_Jlr[0].set_title(fr'Salida vs Valor predicho')
    line2, = axs_Jlr[0].plot(x,plot_h(x,theta0_slider.value,theta1_slider.value),'o',label = r'$h$')
    axs_Jlr[0].plot(x,y,'*', label = r'$y$')
    axs_Jlr[0].set_xlabel('x')
    axs_Jlr[0].grid(True)
    axs_Jlr[0].legend()
                          
    axs_Jlr[1].set_title(fr'Comparación funciones de costo $J(\theta)$ con $\theta_1=1$')
    axs_Jlr[1].plot(theta0, J1, label='Función de costo regresión lineal')
    axs_Jlr[1].plot(theta0, J2, label='Función de costo regresión logística')
    axs_Jlr[1].set_xlabel(r'$\theta_0$')
    axs_Jlr[1].set_ylabel(r'$J(\theta)$')
    axs_Jlr[1].grid(True)
    axs_Jlr[1].legend()
    

def update_plot(change):
    line2.set_ydata(plot_h(x,theta0_slider.value,theta1_slider.value))
    
theta0_slider.observe(update_plot,'value')
theta1_slider.observe(update_plot,'value')

widgets.VBox([plots_Jlr,theta0_slider,theta1_slider],layout = layout)


<h3>CLASIFICACIÓN MULTICLASE</h3>
Cuando el problema de clasificación es multiclase, la salida 
puede tomar n+1 valores ($y \in \{0,1,...,n\}$). En este caso 
se divide el problema en <b>n+1 problemas de clasificación 
binaria</b>. En cada uno de ellos se predice la probabilidad 
de que <b>y</b> pertenezca a cada una de las clases y luego 
se selecciona la clase que maximiza este valor.
<br>    
<ul>   
     <li> Se selecciona una clase y se agrupan todas las otras en 
         una única segunda clase.
        $$
            h_\theta^0 = P(y=0|x;\theta)
        $$</li>
     <li> Se repite lo mismo seleccionando una clase distinta cada vez.
         $$
            h_\theta^1 = P(y=1|x;\theta)
         $$
         $$
            ...
         $$
         $$
            h_\theta^n = P(y=n|x;\theta)
         $$</li>
     <li> Se selecciona la hipótesis que produjo el mayor valor como predicción.
         $$
             prediction= \max_{i}(h_\theta^i(x))
         $$</li>
    </ul>

<center>
<img src="./figures/oneVsAll.png" height="600" width="600"/>
</center>

<hr>
<h3>REGULARIZACIÓN</h3>
<strong>REGRESIÓN LOGÍSTICA CON REGULARIZACIÓN</strong>
<dl> 
    <dt>FUNCIÓN DE COSTO</dt>
    <dd>
        $$
            J(\theta)=-\frac{1}{m}\sum_{i=1}^m\left[y^i\log(h_\theta(x^i))+(1-y^i)\log(1-h_\theta(x^i))\right]+\frac{\lambda}{2m}\sum_{j=1}^n\theta_j^2
        $$
        $$h_\theta(x)=\frac{1}{1+e^{-\theta^T x}}$$
        El término $\sum_{j=1}^n\theta_j^2$ excluye explícitamente al término $\theta_0$
    </dd>
    <br>
    <dt>GRADIENTE DESCENDENTE</dt>
      <dd>Repetir hasta la convergencia:
           $$
               \theta_0 := \theta_0 - \alpha\frac{1}{m}\sum_{i=1}^m \left(h_\theta(x^i)-y^i\right)x_0^i
           $$
           $$
               \theta_j := \theta_j- \alpha\left[\frac{1}{m}\sum_{i=1}^m \left(\left(h_\theta(x^i)-y^i\right)x_j^i\right)+\frac{\lambda}{m}\theta_j\right] \quad j\in \text\{1,2,...,n\text\}
           $$
        </dd>
</dl>
